In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
data = pd.read_csv('data/hll.csv')
data['estimated/n'] = data['expected'] / data['n']
# 130k - hash, 10k - b

,hash,b,n,expected,estimated,estimated/n
1,md5,4,2,2,2,1.0
2,md5,4,3,3,3,1.0
3,md5,4,4,4,4,1.0
4,md5,4,5,5,4,1.0
5,md5,4,6,6,7,1.0
...,...,...,...,...,...,...
129995,md5,16,9996,9996,10029,1.0
129996,md5,16,9997,9997,10040,1.0
129997,md5,16,9998,9998,9998,1.0
129998,md5,16,9999,9999,10041,1.0


In [26]:
summary = data.groupby(['hash', 'b']).agg([np.mean])
summary

n expected  estimated estimated/n
             mean     mean       mean        mean
hash   b                                         
md5    4   5000.5   5000.5  5003.0247         1.0
       5   5000.5   5000.5  4994.2213         1.0
       6   5000.5   5000.5  4987.7134         1.0
       7   5000.5   5000.5  4994.6747         1.0
       8   5000.5   5000.5  4999.9425         1.0
       9   5000.5   5000.5  5003.3717         1.0
       10  5000.5   5000.5  5003.2283         1.0
       11  5000.5   5000.5  5019.4645         1.0
       12  5000.5   5000.5  5003.2192         1.0
       13  5000.5   5000.5  5001.0736         1.0
       14  5000.5   5000.5  5000.5538         1.0
       15  5000.5   5000.5  4999.7100         1.0
       16  5000.5   5000.5  4999.9378         1.0
sha1   4   5000.5   5000.5  5019.3546         1.0
       5   5000.5   5000.5  5001.9598         1.0
       6   5000.5   5000.5  4996.4860         1.0
       7   5000.5   5000.5  4997.6575         1.0
       8   5000.5   5000.5  5001.5595         1.0
       9   5000.5   5000.5  5003.7927         1.0
       10  5000.5   5000.5  5004.3446         1.0
       11  5000.5   5000.5  5019.0118         1.0
       12  5000.5   5000.5  5004.5455         1.0
       13  5000.5   5000.5  5000.6486         1.0
       14  5000.5   5000.5  5000.4374         1.0
       15  5000.5   5000.5  4999.8431         1.0
       16  5000.5   5000.5  4999.6869         1.0
sha256 4   5000.5   5000.5  4997.5863         1.0
       5   5000.5   5000.5  5007.5714         1.0
       6   5000.5   5000.5  5004.9155         1.0
       7   5000.5   5000.5  5006.2382         1.0
       8   5000.5   5000.5  4999.6084         1.0
       9   5000.5   5000.5  4995.7786         1.0
       10  5000.5   5000.5  5004.4395         1.0
       11  5000.5   5000.5  5019.8008         1.0
       12  5000.5   5000.5  5004.2218         1.0
       13  5000.5   5000.5  5000.5282         1.0
       14  5000.5   5000.5  5000.0952         1.0
       15  5000.5   5000.5  4999.5989         1.0
       16  5000.5   5000.5  5000.1563         1.0
sha512 4   5000.5   5000.5  4986.4129         1.0
       5   5000.5   5000.5  4996.0443         1.0
       6   5000.5   5000.5  5002.7611         1.0
       7   5000.5   5000.5  5008.7762         1.0
       8   5000.5   5000.5  5008.2073         1.0
       9   5000.5   5000.5  5002.4680         1.0
       10  5000.5   5000.5  5006.4620         1.0
       11  5000.5   5000.5  5020.3923         1.0
       12  5000.5   5000.5  5002.4599         1.0
       13  5000.5   5000.5  5000.8235         1.0
       14  5000.5   5000.5  5000.3388         1.0
       15  5000.5   5000.5  5000.0734         1.0
       16  5000.5   5000.5  4999.9527         1.0